# Terminating Conversations Between Agents

In this chapter, we will explore how to terminate a conversation between AutoGen agents.

_But why is this important?_ Its because in any complex, autonomous workflows it's crucial to know when to stop the workflow. For example, when the task is completed, or perhaps when the process has consumed enough resources and needs to either stop or adopt different strategies, such as user intervention. So AutoGen natively supports several mechanisms to terminate conversations.

How to Control Termination with AutoGen?
Currently there are two broad mechanism to control the termination of conversations between agents:

1. **Specify parameters in `initiate_chat`**: When initiating a chat, you can define parameters that determine when the conversation should end.

2. **Configure an agent to trigger termination**: When defining individual agents, you can specify parameters that allow agents to terminate of a conversation based on particular (configurable) conditions.


## Parameters in `initiate_chat`
In the previous chapter we actually demonstrated this when we used the `max_turns` parameter to limit the number of turns. If we increase `max_turns` to say `3` notice the conversation takes more rounds to terminate:

In [1]:
import os

from autogen import ConversableAgent
from dotenv import load_dotenv
load_dotenv('credentials.env') 

config_list = [
    {
        "model":  os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'],
        "api_key": os.environ['AZURE_OPENAI_API_KEY'],
        "azure_endpoint": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": "azure",
        "api_version": "2024-02-15-preview"
    }
]

In [2]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={
        "cache_seed": None,
        "config_list": config_list,
        "max_tokens": 1024,
        "temperature": 0.9
    },
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={
        "cache_seed": None,
        "config_list": config_list,
        "max_tokens": 1024,
        "temperature": 0.7
    },
    human_input_mode="NEVER",  # Never ask for human input.
)

In [3]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, Joe! Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
joe (to cathy):

Good one, Cathy! I've got one for you. Why did the scarecrow win an award?

Because he was outstanding in his field!

--------------------------------------------------------------------------------
cathy (to joe):

Nice one, Joe! That’s a classic! Here’s another for you: Why did the bicycle fall over?

Because it was two-tired!

--------------------------------------------------------------------------------


In [5]:
result = joe.initiate_chat(
    cathy, message="Cathy, tell me a joke.", max_turns=3
)  # increase the number of max turns before termination

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, Joe! Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! Speaking of making things up, remember that time we tried to improvise a whole comedy skit on the spot?

Cathy: Oh, how could I forget? It was a mix of brilliance and total disaster!

Joe: Mostly disaster, but hey, at least we made people laugh!

--------------------------------------------------------------------------------
cathy (to joe):

Cathy: Exactly! Remember the bit where you pretended to be a clueless tourist asking for directions?

Joe: Oh, I was like, "Excuse me, which way to the Eiffel Tower?" and we were in the middle of New York City!

Cathy: And I said, "Sure, just take a left at the Statue of Liberty, then a right at the Empir

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:1461: UserWarning: Extracted_response from ChatCompletion(id='chatcmpl-AQbo68A6YFckVfmRZ5AIywWRpzgtt', choices=[Choice(finish_reason='content_filter', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}, logprobs=None)], created=1730905222, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_36b0c83da2', usage=CompletionUsage(completion_tokens=112, prompt_tokens=266, total_tokens=378), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered':


## Agent-triggered termination
You can also terminate a conversation by configuring parameters of an agent.
Currently, there are two parameters you can configure:

1. `max_consecutive_auto_reply`: This condition triggers termination if the number of automatic responses to the same sender exceeds a threshold. You can customize this using the `max_consecutive_auto_reply` argument of the `ConversableAgent` class. To accomplish this the agent maintains a counter of the number of consecutive automatic responses to the same sender. Note that this counter can be reset because of human intervention. We will describe this in more detail in the next chapter.
2. `is_termination_msg`: This condition can trigger termination if the _received_ message satisfies a particular condition, e.g., it contains the word "TERMINATE". You can customize this condition using the `is_terminate_msg` argument in the constructor of the `ConversableAgent` class.

### Using `max_consecutive_auto_reply`

In the example below lets set `max_consecutive_auto_reply` to `1` and notice how this ensures that Joe only replies once.

In [7]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={
        "cache_seed": None,
        "config_list": config_list,
        "max_tokens": 1024,
        "temperature": 0.7
    },
    human_input_mode="NEVER",  # Never ask for human input.
    max_consecutive_auto_reply=1,  # Limit the number of consecutive auto-replies.
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.")

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, Joe! Here's one for you:

Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
joe (to cathy):

Good one, Cathy! Here’s one from me:

Why don’t skeletons fight each other?

They don’t have the guts!

--------------------------------------------------------------------------------
cathy (to joe):

Cathy:
Good one, Joe! You're really cracking me up. But seriously, those skeletons are just spineless!

--------------------------------------------------------------------------------


### Using `is_termination_msg`

Let's set the termination message to "GOOD BYE" and see how the conversation terminates.

In [8]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={
        "cache_seed": None,
        "config_list": config_list,
        "max_tokens": 1024,
        "temperature": 0.7
    },
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to joe):

Sure thing, Joe! Here goes:

Why don’t scientists trust atoms?

Because they make up everything!

GOOD BYE!

--------------------------------------------------------------------------------



Notice how the conversation ended based on contents of cathy's message!

## Summary

In this chapter we introduced mechanisms to terminate a conversation between agents.
You can configure both parameters in `initiate_chat` and also configuration of agents.

That said, it is important to note that when a termination condition is triggered,
the conversation may not always terminate immediately. The actual termination
depends on the `human_input_mode` argument of the `ConversableAgent` class.
For example, when mode is `NEVER` the termination conditions above will end the conversations.
But when mode is `ALWAYS` or `TERMINATE`, it will not terminate immediately.
We will describe this behavior and explain why it is important in the next chapter.
